# Олимпиада НТИ - как выиграть за 30 строк

![](https://pp.userapi.com/c638719/v638719005/2f6da/mFHY83I9u2k.jpg)

### Машинное обучение и анализ данных - задача №1


Описание предметной области. Выявление информации из текста - одна из самых сложных и интересных задач машинного обучения. Речь идет не о разборе предложения или структуре текстов, а о таких вопросах как определения авторства или стиля. Говоря о машинном обучении неизбежно встает вопрос о том, как программе обрабатывать и находить новую информацию в огромных объемах данных, которые даже человек не может осмыслить. Например, нет такого в мире специалиста, который бы мог точно определить авторство любой заметки из 200 авторов начала 20го века. Возможность обучения без учителя, когда вручную отмечаются правильные ответы лишь на небольшом количестве данных - одна из самых актуальных задач машинного обучения. Ведь как только алгоритмы смогут эффективно обучаться на больших объемах без предварительной ручной разметки создание сильного искусственного интеллекта будет сильно ближе.

Описание актуальной задачи. В качестве данных предлагается анализировать личные дневники 19го и начала 20го века. Настоящее авторство текстов - популярный вопрос среди искусствоведов, дискуссии о котором не утихают десятилетиями. Благодаря аналитике больших данных можно не просто сказать к чьему перу относится запись, но и выявить под чей стиль она подходит и выявить важные факты, передав искусствоведам эффективный инструмент анализа. Однако, возможен случай когда у набора записей не осталось сведения об авторах, и список авторов неограничен. Чтобы разобрать эти записи можно применять кластеризацию, и находить схожие заметки. При этом, в обучающей выборке лишь небольшое количество размеченных записей (около 2000), и большое (десятки тысяч) неразмеченных данных. Необходимо реализовать обучение с частичным привлечением учителя (semi-supervised learning).

Всего на сайте около 200 авторов, и 100 000 записей дневников.

В качестве обучающей выборки представлено 100 000 записей, из которых 2.000 будет отмечено авторство.

## Решение

Задача содержит 3 датасета:

Первые два даются участникам, а на основе третьего генерируется score.

Давайте скачаем эти датасеты.

In [1]:
!wget https://tvorog.me/task1_test.csv

--2017-04-06 13:14:16--  https://tvorog.me/task1_test.csv
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся tvorog.me… 88.212.244.196
Подключение к tvorog.me|88.212.244.196|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 200074643 (191M) [application/octet-stream]
Сохранение в: «task1_test.csv»

task1_test.csv      100%[===================>] 190,81M  7,82MB/s    за 19s     

2017-04-06 13:14:35 (10,3 MB/s) - «task1_test.csv» сохранён [200074643/200074643]



In [2]:
!wget https://tvorog.me/task1_train.csv

--2017-04-06 13:14:46--  https://tvorog.me/task1_train.csv
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся tvorog.me… 88.212.244.196
Подключение к tvorog.me|88.212.244.196|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 204840156 (195M) [application/octet-stream]
Сохранение в: «task1_train.csv»

task1_train.csv     100%[===================>] 195,35M  7,32MB/s    за 28s     

2017-04-06 13:15:15 (6,95 MB/s) - «task1_train.csv» сохранён [204840156/204840156]



## Датасет с ответами.
Его мы используем только для проверки ответов

In [3]:
!wget https://tvorog.me/task1_answers.csv

--2017-04-06 13:15:16--  https://tvorog.me/task1_answers.csv
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся tvorog.me… 88.212.244.196
Подключение к tvorog.me|88.212.244.196|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1077811 (1,0M) [application/octet-stream]
Сохранение в: «task1_answers.csv»

task1_answers.csv   100%[===================>]   1,03M  3,72MB/s    за 0,3s    

2017-04-06 13:15:17 (3,72 MB/s) - «task1_answers.csv» сохранён [1077811/1077811]



# Посмотрим на данные

[Pandas](http://pandas.pydata.org/) это бог табличной PyData. Неплохие примеры его использования можно найти [тут](http://pandas.pydata.org/pandas-docs/stable/tutorials.html).

In [1]:
import pandas as pd

In [2]:
# Pandas позволяет очень просто грузить csv
test = pd.DataFrame.from_csv("task1_test.csv")

In [3]:
# .head() показывает первые 5 элементов таблицы
test.head()

,text,person_id
note_id,,
198020,В Ленинград<ском> доме ученых (б<ывший> дворец...,-1
197857,Мы приехали часов в 12. Т. Ворошилов встретил ...,-1
231988,1 May 1663. \nUp betimes and my father with...,-1
184624,После полудня генерал послал меня вместе с бар...,-1
85563,"Воскресенье. Когда вышел, чтобы поехать обедат...",-1


In [4]:
train = pd.DataFrame.from_csv("task1_train.csv")

In [5]:
train.head()

,text,person_id
note_id,,
933,"Вечер, 20.30. Из города после страшной спешки ...",4
910,"Просто удивительно. Прошло уже десять дней, а ...",4
909,Сегодня на дачу переезжает старшая дочь Наташа...,4
908,Много работал над романом. После обеда Гера уе...,4
907,День рождения дочери Лены. \nВстал я утром ка...,4


К элементам таблицы можно обращаться двумя способами.

1. train['text']
2. train.text

Они возвращают запрашиваемый столбец. Единственная загвоздка заключается в index колонке. 

train.note_id
```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-26-ed64215d46cd> in <module>()
----> 1 train.note_id

/home/tvorog/.anaconda3/lib/python3.6/site-packages/pandas/core/generic.py in __getattr__(self, name)
   2742             if name in self._info_axis:
   2743                 return self[name]
-> 2744             return object.__getattribute__(self, name)
   2745 
   2746     def __setattr__(self, name, value):

AttributeError: 'DataFrame' object has no attribute 'note_id'```



In [6]:
train.index

Int64Index([  933,   910,   909,   908,   907,   906,   905,   904,   903,
              902,
            ...
            51277, 51275, 51274, 51273, 51276, 51271, 51270, 51269, 51268,
            51272],
           dtype='int64', name='note_id', length=95087)

In [7]:
# Counter считает количество объектов в итераторе и возвращает tuple (item: count)
from collections import Counter

In [8]:
# .most_common() сортирует по count
Counter(train.person_id).most_common()

[(213, 6603),
 (223, 3570),
 (175, 3535),
 (129, 3083),
 (56, 2862),
 (183, 2595),
 (134, 2462),
 (163, 2213),
 (151, 2182),
 (39, 1941),
 (194, 1898),
 (229, 1829),
 (102, 1631),
 (15, 1358),
 (188, 1333),
 (115, 1314),
 (57, 1190),
 (179, 1167),
 (254, 1164),
 (78, 1045),
 (117, 1040),
 (21, 1028),
 (48, 1008),
 (16, 905),
 (30, 880),
 (189, 872),
 (22, 868),
 (126, 865),
 (25, 856),
 (138, 842),
 (11, 804),
 (209, 783),
 (198, 779),
 (53, 775),
 (146, 752),
 (42, 715),
 (131, 690),
 (17, 684),
 (258, 675),
 (260, 672),
 (178, 668),
 (135, 660),
 (27, 636),
 (83, 635),
 (118, 627),
 (249, 619),
 (36, 608),
 (87, 586),
 (34, 584),
 (13, 568),
 (212, 567),
 (26, 550),
 (294, 522),
 (18, 514),
 (157, 512),
 (203, 508),
 (66, 506),
 (199, 493),
 (50, 478),
 (67, 474),
 (149, 451),
 (225, 436),
 (173, 425),
 (210, 414),
 (103, 404),
 (214, 398),
 (197, 393),
 (298, 389),
 (88, 387),
 (136, 384),
 (110, 376),
 (90, 369),
 (200, 366),
 (37, 363),
 (5, 356),
 (274, 336),
 (4, 329),
 (93, 323

In [9]:
Counter(test.person_id).most_common()

[(-1, 59445),
 (629, 201),
 (402, 201),
 (566, 201),
 (454, 201),
 (486, 201),
 (374, 201),
 (600, 201),
 (491, 201),
 (616, 201),
 (418, 201),
 (328, 201),
 (505, 201),
 (535, 201),
 (416, 201),
 (429, 201),
 (615, 201),
 (367, 201),
 (428, 201),
 (527, 201),
 (450, 201),
 (528, 201),
 (543, 201),
 (668, 201),
 (453, 201),
 (312, 201),
 (441, 201),
 (426, 201),
 (999, 201),
 (455, 201),
 (635, 201),
 (565, 201),
 (434, 201),
 (497, 201),
 (405, 201),
 (362, 201),
 (448, 201),
 (372, 201),
 (327, 201),
 (369, 201),
 (571, 201),
 (444, 201),
 (423, 201),
 (551, 201),
 (331, 201),
 (382, 201),
 (325, 201),
 (316, 201),
 (649, 201),
 (572, 201),
 (969, 201),
 (536, 201),
 (830, 201),
 (311, 201),
 (433, 201),
 (360, 201),
 (705, 201),
 (711, 201),
 (457, 201),
 (427, 201),
 (556, 201),
 (431, 201),
 (719, 201),
 (960, 201),
 (319, 201),
 (445, 201),
 (523, 201),
 (706, 201),
 (481, 201),
 (349, 201),
 (335, 201),
 (460, 201),
 (323, 201),
 (512, 201),
 (315, 201),
 (840, 201),
 (386, 201)

# Организаторы во всём виноваты

![](https://images5.alphacoders.com/633/633216.jpg)

В задании было сказанно о том, что размеченных авторов примерно 2 тысячи и поэтому нужно использовать semi. Так ли это?

In [10]:
# В pandas можно выбирать элементы с помощью банальных выражений, это все очень похоже на sql
len(test[test.person_id != -1])

40875

# Baseline

Не смотря на то, что организаторы во всём виноваты, мы с вами хотим выйграть олимпиаду НТИ. Сейчас я покажу на сколько просто это сделать. Наш baseline будет состоять из трёх частей. 

1. Чистка текста
2. Векторизация текста
3. Создание и тренировка модели

Использовать мы будем только test датасет, потому что 40к текстов - это вполне нормальная выборка для обучении модели.

### №1 Чистка данных

Для того, чтобы преобразовать все слова в начальную форму достаточно скачать библиотеку [pymorphy2](https://pymorphy2.readthedocs.io/en/latest/). Она распарсит нужные вам слова и приведет их в начальную форму.

In [11]:
import pip
pip.main(['install','pymorphy2'])

0

In [12]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Loading dictionaries from /home/xenx/.anaconda3/lib/python3.6/site-packages/pymorphy2_dicts/data
format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [13]:
# Для того, чтобы исключить все лишние знаки будем использовать regexp
import re

In [14]:
# Уникальных слов, вангую, очень много. Давайте кэшировать.
global_morph = {}

In [15]:
def normalize(text: str) -> str:
    '''Функция, которая принимает строку и возвращает строку. Чистит текст и сводит к начальной форма'''
    global global_morph
    
    # Приведем все в нижний регистр
    text = text.lower()
    
    # Заменим все бяки *не буквы* на пустоту
    text = re.sub(r'\d+', '', text)
    
    # Возьмём все слова
    words = re.findall(r'\w+', text)
    
    tmp = ""
    
    # Для каждого слова
    for word in words:
        # Если оно закешированно
        if word in global_morph.keys():
            # Возьмём его
            tmp += global_morph[word].normal_form
            tmp += " "
        # Иначе
        else:
            # Закешируем
            global_morph[word] = morph.parse(word)[0]
            
            # Возьмём его
            tmp += global_morph[word].normal_form
            tmp += " "

    return tmp[:-1]

In [16]:
# Pandas позволяет применить ко всей колонке функцию.
test.text = test.text.apply(normalize)

## №2 Векторизация

Всемогучий [sklearn](http://scikit-learn.org/stable/). Представьте, если бы была библиотека, которая за вас может выйграть олимпиаду... Так вот, она есть. Именуется sklearn. 

![](http://s9.pikabu.ru/post_img/2016/12/12/8/og_og_1481547098267285367.jpg)

Немного wikipedia:

```
TF-IDF (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален количеству употребления этого слова в документе, и обратно пропорционален частоте употребления слова в других документах коллекции.
```

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()

Для начала нужно взять все слова в текстах и посчитать сколько раз они там встречаются *для того, чтобы это было возможно мы и приводили всё в начальную форму*. К счастью это можно сделать в одну строчку.

In [19]:
vectorizer.fit(test.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

Дальше разобьём датасет на 2 части. В одной части соберём все записи, которые размечены на авторов. Во вротой части будут записи для которых мы должны предсказать.

In [20]:
data_train = test[test.person_id != -1]
data_predict =  test[test.person_id == -1]

Для того, чтобы обучить модель нам нужно разбить всё на две части. Возьмём все тексты и положем их в X, возьмём всех авторов и положем их в y. Но поскольку наш `vectorizer` уже знает все слова в текстах, он может нам дать вектор для каждого текста, состоящий из TF-IDF значений. Давайте мы попросим его это сделать.

In [21]:
X = vectorizer.transform(list(data_train['text']))
y = list(data_train['person_id'])

## №3 Создание и тренировка модели


Поскольку у нас много авторов - это задача регрессии. А если есть задача регрессии - то есть и модель, которая такую задачу решает. И, как не странно, называется она ```LogisticRegression```.

In [22]:
from sklearn.linear_model import LogisticRegression

В ```LogisticRegression``` на результат влияет только одна константа: ```C```. Для улучшения обобщающей способности получающейся модели, то есть уменьшения эффекта переобучения существует регуляризация. ```C``` - это обратная степень регуляризации. Чем меньще ```C``` тем меньше переобучается модель, но тем сложнее обучить модель. 

In [24]:
# n_jobs - кол-во процессеров, которое мы используем
model = LogisticRegression(C=170, n_jobs=-1) # создадим модель с C = 170 *белка подсказала*.

# ВАЖНО! Модель со 170 может обучаться очень долго *20 минут*, так что лучше поставить поменьше.

Обучим модель.

In [25]:
model.fit(X,y)

LogisticRegression(C=170, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Теперь наша модель *обучена*. Это такое великолепное состояние, когда она может нам что-то дать. Но для того, чтобы она нам что-то дала, в неё нужно что-то положить!

In [26]:
X_answer = vectorizer.transform(list(data_predict['text'])) # векторезуем текст
answer = model.predict(X_answer) # и предсказываем по нему

## Done!
#### А что теперь?

Давайте поймём на сколько хороша наша модель.

In [27]:
# Как я уже писал ранее, все ответы лежат в этом *забавном* csv.
answers = pd.DataFrame.from_csv('task1_answers.csv')

Библиотека, которая за вас решает олимпиаду может вам сказать на сколько вы классные. В ```sklearn``` есть реализация ```accuracy_score```. ```accuracy_score``` - это процент того, сколько раз вы угадали.

In [32]:
from sklearn.metrics import accuracy_score

#сложно

Дело в том, что нас просили предсказать для каждого текста в ```task1_test.csv``` автора. А это значит, что авторы, которые уже размечены тоже учитываются.

In [40]:
answers.head()

,person_id
note_id,
198020,536
197857,536
231988,999
184624,719
85563,318


Давайте сопоставим индексы и ответы

In [47]:
train_index_to_answer = {x:y for x,y in zip(data_train.index, data_train['person_id'])}

In [49]:
test_index_to_answer = {x:y for x,y in zip(data_predict.index, answer)}

А теперь сгенерируем полный ответ на задачу.

In [50]:
full_answer = []

for index in answers.index:
    if index in train_index_to_answer.keys():
        full_answer.append(train_index_to_answer[index])
    else:
        full_answer.append(test_index_to_answer[index])

# Ура. 

Посмотрим на ```score```.

![](http://www.setwalls.ru/pic/201305/1680x1050/setwalls.ru-43884.jpg)

In [51]:
accuracy_score(answers.person_id, full_answer)

0.80614035087719293

# Что делать?

1. Можно улучшать текущую модель с помощью sklearn.
2. Можно улучшить обработку данных.
3. Можно сделать настоящий ```semi-supervised learning```. Я сделаю отдельную тетрадку для этого. Вы можете помочь мне и сделать Pull Request в [мой репозиторий](https://github.com/xenx/nti-bigdata2017).
4. Можно попробовать свои идеи для этой задачи, например, реализовать [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) или [SGDClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), а лучше сделать [Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) из этого.

Надеюсь, что данная тетрадка дала вам почву для размышления и обучения. Если у вас есть вопросы - вы всегда можете написть мне:

1. [VK](https://vk.com/tvorogme)
2. [Telegram]()
3. tvorog@tvorog.me